# Assignment 2 :

## Rohit Roy | MDS202340

In [2]:
from tqdm import tqdm
import regex as re
import os
import glob
from nltk import ngrams, word_tokenize, sent_tokenize,pos_tag
import nltk
import random
from collections import Counter, defaultdict

In [ ]:
# Merging into one file

def merge_files():

    input_files = glob.glob(os.path.join("wiki", "*.txt"))
    input_files = sorted(input_files)

    output_file = "process_wiki/wiki.txt"
    with open(output_file, "w", encoding="utf-8") as output:
        for input_file in input_files:
            with open(input_file, "r", encoding="utf-8") as input:
                # Read the content of the input file and write it to the output file
                content = input.read()
                output.write(content)

merge_files()

# Part 1

In [ ]:
# Counting number of sentences.

with open("process_wiki/wiki.txt","r",encoding="utf-8") as input:
    content = input.read()
    sentences = sent_tokenize(content)

num_sent = len(sentences)
print(f"The number of sentences is {num_sent}")

The number of sentences is 7909229


In [ ]:
from collections import defaultdict

# Tokenizing the Corpus into words.
words = [word for sentence in sentences for word in word_tokenize(sentence)]

# Create 4-grams.
four_grams = list(ngrams(words, 4))

In [ ]:
# Now we calculate the probabilities in the list of ngrams.
from collections import Counter

# We create a nested dictionary of words in the corpus with their probability distributions.
ngram_model = defaultdict(Counter)

# For every trigram, there is a separate Counter for each 4th word.
for w1, w2, w3, w4 in four_grams:
    ngram_model[(w1, w2, w3)][w4] += 1

# Converting those counts to probabilities.
for trigram in ngram_model:
    total_count = sum(ngram_model[trigram].values())
    for w4 in ngram_model[trigram]:
        ngram_model[trigram][w4] = ngram_model[trigram][w4] / total_count

In [14]:
trigram_list = [("with", "the", "help"), ("focused", "on", "the"), ("for", "the", "people"), ("have", "ranged", "from"), ("to", "reduce", "their")]

# Now we predict the next word based on the probabilities.
prediction = {}

for trigram in trigram_list:
    next_word_prob = ngram_model[trigram]
    if next_word_prob:
        predicted_word = max(next_word_prob, key=next_word_prob.get)
    else:
        predicted_word = "<UNSEEN>"
    prediction[trigram] = predicted_word

print("The next word that is predicted : ")
prediction

The next word that is predicted : 


{('with', 'the', 'help'): 'of',
 ('focused', 'on', 'the'): 'development',
 ('for', 'the', 'people'): 'of',
 ('have', 'ranged', 'from'): 'narrative',
 ('to', 'reduce', 'their'): 'ecological'}

In [29]:
# Now to generate the sentences using the 4 gram model.

# We use a random start word. So we find all the trigrams that start with that word.
def trigram_getter(word):
    trigram_list = []
    for i in list(ngram_model.keys()):
        if word == i[0]:
            trigram_list.append(i)
    return trigram_list

def sent_generate(start_word):
    start_trigram = random.choice(trigram_getter(start_word))
    list_sent = list(start_trigram)
    for i in range(7):
        next_word_prob = ngram_model[tuple(list_sent[-3:])]
        if not next_word_prob:
            break
        next_word = random.choices(list(next_word_prob.keys()),list(next_word_prob.values()))[0]
        list_sent.append(next_word)
    sentence = " ".join(list_sent)
    return sentence

# Generate 5 sentences using random trigrams as starting points
gen_sent = []

for i in range(5):
    random_start_word = random.choice(words)
    sentence = sent_generate(random_start_word)
    gen_sent.append(sentence)

print("The sentences generated are :")
gen_sent

The sentences generated are :


['for his Canadian landscape scenes . Paul Southern ( born',
 'hockey players : Dillon played in the Greater Brisbane League',
 'by Mad Man Pondo and Necro Butcher . The first',
 'cartoon short , directed by Dr. Balakrishnan and produced by',
 'history to play in the West Godavari district of the']

# Part 2

In [ ]:
# Writing the entire corpus in POS, saving this to a file called pos_wiki.txt

tagged_corpus = []

with open("pos_wiki.txt","w") as file:

    for sent in sentences:
        words = word_tokenize(sent)
        tagged_words = pos_tag(words)
        tagged_corpus.append(tagged_words)
        #print(tagged_words)
        for word in tagged_words:
            file.write(f"{word[1]} ")
        file.write("\n")

In [ ]:
# Finding frequency of each structure.

from collections import Counter

pos_counter = {}

with open("pos_wiki.txt","r") as file:
    for sent in file.readlines():
        sent = sent.strip()
        if sent in pos_counter.keys():
            pos_counter[sent] += 1
        else:
            pos_counter[sent] = 1

In [ ]:
# Number of unique structures.

print(f"Number of unique POS : {len(pos_counter)}")

# Sort the dictionary items by values in descending order and get the top 5 keys.
top_pos = sorted(pos_counter, key=lambda x: pos_counter[x], reverse=True)[:5]

print("Top 5 structures are :")
print(top_pos)

Number of unique POS : 6385780
Top 5 structures are :
['NNP NN VBZ DT NNS IN NN IN DT NN NNP .', 'NNP NN VBZ DT NN IN DT NN NNP .', 'NNP NN VBZ DT NN IN DT NNP NN .', 'NNP NN VBZ DT NNS IN NN IN DT NNP NN .', 'NNP VBZ DT NN IN NNS IN DT NN NNP .']


# Part 3

In [ ]:
# Making a nested dictionary of POS structures.

pos_dict = {}

for sent in tagged_corpus:
    for key, value in sent:
        if value in pos_dict.keys():
            if key in (pos_dict[value]).keys():
                pos_dict[value][key]+=1
            else:
                pos_dict[value][key]=1
        else:
            pos_dict[value] = {key: 1}

In [ ]:
# Making probability distribution for each POS, and generating sentences for top_pos.

pos_prob_dict={}

for x in pos_dict.keys():
    pos_prob_dict[x]=dict(sorted((pos_dict[x]).items(), key=lambda x:x[1], reverse=True))

for w in pos_prob_dict:
    total_count = sum(pos_prob_dict[w].values())
    for word in pos_prob_dict[w]:
        pos_prob_dict[w][word] = pos_prob_dict[w][word] / total_count

random.seed(17)

for i in range(len(top_pos)):
    print(f"Structure {i+1}:")
    print(top_pos[i])
    for j in range(5):
        print(f"\n {j+1} .", end = " ")
        for j in range(len(list(top_pos[i].split()))):
            pos_list = list(top_pos[i].split())
            next_word = random.choices(list(pos_prob_dict[pos_list[j]].keys()), weights=list(pos_prob_dict[pos_list[j]].values()), k=1)
            print(next_word[0], end=" ")
    print("\n")

Structure 1:
NNP NN VBZ DT NNS IN NN IN DT NN NNP .

 1 . Southwest insectivore is the stories of planning of the city Santa . 
 2 . Frank cell is a donors of passenger of the nomenclature College . 
 3 . January rodent has the dependents of dash in a album Works . 
 4 . Committee album is the crowds through harmony from The list Australia . 
 5 . Policy reality is a people for moth by the appeal Comedy . 

Structure 2:
NNP NN VBZ DT NN IN DT NN NNP .

 1 . Stabber team chooses This steel of the practice Munich . 
 2 . Essex hurdler is the production at the program State . 
 3 . Lough heritage plays the foliage In the photographs Lake . 
 4 . Vaine constituency is the component in the midday Computer . 
 5 . Office musician is The section from a place Ronald . 

Structure 3:
NNP NN VBZ DT NN IN DT NNP NN .

 1 . City population plays a logo in the A. county . 
 2 . Jersey plank is the instruction in an Punjab research . 
 3 . Invest case is an series in the Nigerian dementia . 
 4 . Mi